This is a retrospective data analysis of our ESPN Fantasy Football league data at the end of Week 17 for the 2022 season. The code can be re-used for any past season using the same ESPN API.

I used the [Pro Football Reference](https://www.pro-football-reference.com/years/2022/games.htm) for NFL schedules and [Christian Wendt's ESPN Fantasy Football API](https://github.com/cwendt94/espn-api) to extract ESPN league-specific data.

For data visualization, I used the [Altair Data Visualization Library](https://altair-viz.github.io/). Altair was chosen for its interactivity, especially for its ability to display hover data and the compatibility of interactive plots with GitHub Pages.

One question this analysis can answer is: what was the return on investment of a given players with fantasy draft or waiver budget dollars (FAAB).

See src directory for source code.

In [1]:
# %cd /Users/jonathancheng/PycharmProjects/espnff/src/
# %cd ../src/espnff_analysis/


In [2]:
# pwd

In [3]:
import streamlit as st

import espnff_analysis.nfl_schedule as nf
import espnff_analysis.ff_league_data as ff
import espnff_analysis.calc_best_waiver as cbw
from espnff_analysis.plotting import scatterplot_acquisitions




## Get NFL Schedule

In [4]:
year_of_interest=2020

path = r'/Users/jonathancheng/PycharmProjects/espnff/espnff_analysis/data'

league_id = 1094090
year = year_of_interest

swid = "{F191FB8C-DB2D-4D24-91FB-8CDB2DED249D}"
s2='AECJMQHsUHB0FTXdZkw93uY7GRbX8BPnm93Ye6AwvwrMsrZFGg1Lbmi07SWVov2ioN8zGMFDzZiiDSeQCa7WQHaGivGnMfGWLjmfGwkOeLXb5baD1sltp%2B%2BIfHAtl98TpmHgB16ZpGn6g3Bm5vLEA7yDC6HkbD3LSp0E2rGB7hKziLMvZ7mT6ONJFRe8Xp3ApYWSvxPr9cz0pJiI%2FF0blsZ8hyATDJMEyaQ2O%2FypcsViORr6hqYTmXHPuPKnMBfvYC8LQqi1exGw3vnyg6ptsB2Y'

espn_s2 = s2


In [5]:
df_proteam_schedule = nf.get_nfl_schedule(year_of_interest)
season_start_date = nf.get_season_start_date(df_proteam_schedule)

## Generate League object

In [6]:
league = ff.fetch_espn_api(league_id, year, espn_s2, swid)


In [13]:
import retry
import requests

@retry.retry(tries=5,delay=3)
def get_league_activity(league , n_iter=1000000):
    return league.recent_activity(n_iter)

def get_league_activity_wrapper(league):
    try:
        return get_league_activity(league)
    except requests.exceptions.ConnectionError:
        raise ConnectionError ("Could not fetch league data, try re-running the program.")
     

activity_ls = get_league_activity_wrapper(league)

ConnectionError: Could not fetch league data, try re-running the program.

ConnectionError: ('Connection aborted.', TimeoutError(60, 'Operation timed out'))

In [ ]:
activity_ls = league.recent_activity(1000000)
wk_ls = ff.get_weeks(league)

In [ ]:
import time

max_attempts = 3  # set the maximum number of attempts
attempt = 1  # initialize the attempt counter

while attempt <= max_attempts:
    try:
        # code that might time out goes here
        # ...
        # if the code runs successfully, break out of the loop
        activity_ls = league.recent_activity(1000000)

        break
    except Exception as e:
        # if there's an exception (e.g. a timeout), print an error message
        print(f"Error: {e}")
        # wait for a bit before retrying
        time.sleep(5)
        # increment the attempt counter
        attempt += 1
else:
    # if the maximum number of attempts is reached without success, print an error message
    print("Maximum number of attempts reached. Program timed out.")


## Get Acquisitions Data

In [ ]:
# fetch league data, wrangle into acquisitions DataFrame

acq_data_flat_ls = ff.get_acq_ls(activity_ls)

df_acq = ff.build_df_acq(acq_data_flat_ls)

## Get Draft Data

In [ ]:
df_draft,drafted_players = ff.build_df_draft(league)

## Get total points of rostered players Dataframe

In [ ]:
df_rostered = ff.build_df_rostered(league)

## Get total points of free agent players Dataframe

In [ ]:
df_FA = ff.build_df_FA(league)

In [ ]:
# Generate all player stats dataframe, including all Free Agents
df_player_stats = ff.build_df_player_stats(df_rostered,df_FA)

In [ ]:
df_draft_stats = ff.build_df_draft_stats(df_draft,df_player_stats)
df_acq_stats = ff.build_df_acq_stats(df_acq,df_player_stats)
df_acq_final = ff.build_df_acq_final(season_start_date, df_draft_stats, df_acq_stats, drafted_players)

## Get player_box_scores from fantasy season

In [ ]:
df_player_box_scores = ff.build_df_player_box_scores(league, wk_ls)

## Construct df_stints

In [ ]:
df_stints=ff.build_df_stints(df_acq_final, df_proteam_schedule, df_player_stats, drafted_players, df_player_box_scores)

In [ ]:

df_stints['Total points per stint'] = df_stints.apply(lambda x: ff.get_total_pts_per_player(x['Player'], x['Stint (wks)'], df_player_box_scores),axis=1).fillna(0)

In [ ]:
def merge_lists(series):
    return list(set([item for sublist in series for item in sublist]))
    # return list(itertools.chain(*ls))

In [ ]:
df_temp=df_stints.groupby(by=['Player', 'Team']).agg({'Total points per stint':'sum', 'Stint (wks)':list}).reset_index()

In [ ]:
df_temp['Stint (wks)'] = df_temp['Stint (wks)'].apply(lambda x: list(itertools.chain(*x)))

In [ ]:
df_temp

In [ ]:
df_stints[['Player','Team','ProTeam','Position','Drafted']].drop_duplicates()

In [ ]:
df_player_ffteam = df_temp.merge(df_stints[['Player','Team','ProTeam','Position','Drafted']].drop_duplicates(), how='left')

In [ ]:
for idx, x in df_player_ffteam.iterrows():
    if x['Stint (wks)']:
        df_player_ffteam.loc[idx,'quantile'] = cbw.calculate_scoring_quantile_per_stint(x['Stint (wks)'], x['Position'], x['Total points per stint'],df_player_box_scores)
    else:
        df_player_ffteam.loc[idx,'quantile'] = 0
        

In [ ]:
df_player_ffteam['Num weeks'] = df_player_ffteam['Stint (wks)'].apply(lambda x: len(x))

In [ ]:
df_player_ffteam

In [ ]:
df_waiver = df_player_ffteam[(~df_player_ffteam['Drafted']) & (df_player_ffteam['Num weeks']>=8)]

In [ ]:
df_waiver.sort_values('quantile',ascending=False)

In [ ]:
df_stints.head()

In [ ]:
for idx, x in df_stints.iterrows():
    if x['Stint (wks)']:
        df_stints.loc[idx,'quantile'] = cbw.calculate_scoring_quantile_per_stint(x['Stint (wks)'], x['Position'], x['Total points per stint'],df_player_box_scores)
    else:
        df_stints.loc[idx,'quantile'] = 0
        

In [ ]:
df_stints

In [ ]:
plot_title, chart=scatterplot_acquisitions(df_stints, select_acq_method=[True], select_positions=['QB','RB', 'WR', 'TE'])
chart.display()

In [ ]:
import altair as alt
import pandas as pd

select_acq_method=[True] 
select_positions=['QB','RB', 'WR', 'TE']

if select_acq_method is None:
    select_acq_method = [True]

if select_positions is None:
    select_positions = ['RB', 'WR', 'TE']

g = df_stints.groupby(by=["Drafted", "Position"])

df = pd.concat([g.get_group((acq_by_draft, position))
                for acq_by_draft in select_acq_method
                for position in select_positions], axis=0)

if select_acq_method[0]:
    status = "Draft"
else:
    status = "Waiver"

positions = ', '.join(select_positions)

plot_title = f"Position: {positions} , Acquired by: {status}"
selection = alt.selection_multi(fields=["Team"], bind="legend")

color = alt.condition(
    selection,
    alt.Color(
        "Team:N",
        scale=alt.Scale(scheme="tableau20"),
    ),
    alt.value("lightgray"),
)

chart = (
    alt.Chart(df)
    .mark_circle(size=40)
    .encode(
        alt.X("Bid Amount ($)", axis=alt.Axis(grid=False)),
        alt.Y("quantile", axis=alt.Axis(grid=False)),
        color=color,
        opacity=alt.condition(selection, alt.value(1), alt.value(0.1)),
        tooltip=["Player", "Team", "Bid Amount ($)", "Total points per stint"],
    )
    .add_selection(selection)
    .properties(width=450, height=450, title=plot_title)
    .configure_axis(labelFontSize=18, titleFontSize=18)
    .configure_title(fontSize=20)
    .configure_legend(labelFontSize=14, titleFontSize=14)
)

In [ ]:
chart.display()

In [ ]:
import itertools
list(itertools.chain(*ls))

In [ ]:
# Build a scoring algorithm 
# Input: Position, Stint (wks), aTotal points, num weeks played
# Calculate that player's percentile against the rest of his position for the total points scored in the given weeks
# Get 1D array of the particular position within the timespan

In [ ]:
def get_total_pts_per_player(player, stint, df_player_box_scores):
    if stint:
        g = df_player_box_scores.groupby(by="Player")
        df = g.get_group(player)
        return df[df["Week"].isin(stint)]["Total points"].sum()

In [ ]:
# get quantile per stint



In [ ]:
df_stints[df_stints['Player'].str.contains('Daniel Jones')]

In [ ]:
df_stints.head()

In [ ]:
stint = [13, 14, 15, 16, 17, 18]
position = 'QB'
total_points_oneplayer = 102.9

df_temp=df_player_box_scores[(df_player_box_scores['Position'] == position) & (df_player_box_scores['Week'].isin(stint))]
total_position_stint = df_temp.groupby(by=['Player']).agg({'Total points':'sum'}).reset_index()['Total points']
quantile = total_position_stint.quantile((total_points_oneplayer-total_position_stint.min())/(total_position_stint.max()-total_position_stint.min()))


In [ ]:
from scipy import stats

def calculate_scoring_quantile_per_stint(stint,position,total_points_oneplayer):
    
    df_allplayers_stint = df_player_box_scores[(df_player_box_scores['Position'] == position) & (df_player_box_scores['Week'].isin(stint))]
    total_position_stint = df_allplayers_stint.groupby(by=['Player']).agg({'Total points':'sum'}).reset_index()['Total points']
    quantile = stats.percentileofscore(total_position_stint.values, total_points_oneplayer)
    return quantile


In [ ]:
df_stints=df_stints.drop(['quantile'],axis=1)

In [ ]:
for idx, x in df_stints.iterrows():
    if x['Stint (wks)']:
        df_stints.loc[idx,'quantile'] = calculate_scoring_quantile_per_stint(x['Stint (wks)'], x['Position'], x['Total points per stint'])
    else:
        df_stints.loc[idx,'quantile'] = 0
    

In [ ]:
df_stints['Num wks'] = df_stints['Stint (wks)'].apply(lambda x: len(x))

In [ ]:
df_stints1 = df_stints[df_stints['Num wks']>=7]

In [ ]:
df_stints1[~df_stints1['Drafted']].sort_values(by=['quantile'],ascending=False)

In [ ]:
for idx,x in df_stints.iterrows():
    if idx == 1:
        break

In [ ]:
x

In [ ]:
stint = x['Stint (wks)']
position = x['Position']
total_points_oneplayer = x['Total points per stint']
print(total_points_oneplayer)

df_allplayers_stint = df_player_box_scores[(df_player_box_scores['Position'] == position) & (df_player_box_scores['Week'].isin(stint))]


In [ ]:
total_position_stint = df_allplayers_stint.groupby(by=['Player']).agg({'Total points':'sum'}).reset_index()['Total points']


In [ ]:
total_position_stint.max()

In [ ]:
total_position_stint.min()

In [ ]:
quantile = stats.percentileofscore(total_position_stint.values, total_points_oneplayer)


In [ ]:
quantile

In [ ]:
# quantile = total_position_stint.quantile((total_points_oneplayer-total_position_stint.min())/(total_position_stint.max()-total_position_stint.min()))

In [ ]:
quantile

In [ ]:
x

In [ ]:
df_stints[df_stints['Drafted']==False].sort_values(by=['quantile'],ascending=False)

In [ ]:
# df_stints1 = df_stints[df_stints['Player'].str.contains('Daniel Jones')].iloc[-1,:]
# df_stints1.apply(lambda x: calculate_scoring_quantile_per_stint(x['Stint (wks)'],x['Position'],x['Total points per stint']),axis=1)
df_stints1.apply(lambda x: calculate_scoring_quantile_per_stint(x['Stint (wks)'], x['Position'], x['Total points per stint']), axis=1)


In [ ]:
a = df_stints[(df_stints['Player'].str.contains('Daniel Jones'))].iloc[-1,:]['Total points per stint']

In [ ]:
a

In [ ]:
stint

In [ ]:
18 in stint

In [ ]:
import pandas as pd

df_temp = pd.concat([df_temp.groupby(by='Week').get_group(i) for i in stint if i!=18])
df_temp['Total points'].sum()/df_temp.shape[0]

In [ ]:
s=df_temp['Total points']

In [ ]:
df_temp.groupby(by=['Player']).get_group('Daniel Jones')

In [ ]:
df_temp.groupby(by=['Player']).get_group('Tom Brady')['Total points'].sum()

In [ ]:
df_temp['Total points']

In [ ]:
df_temp=df_temp.groupby(by=['Player']).agg(sum).reset_index().sort_values(by='Total points', ascending=False)

In [ ]:
s = df_temp['Total points']


In [ ]:
quantile = s.quantile((a-s.min())/(s.max()-s.min()))


In [ ]:
df_temp

In [ ]:
quantile

In [ ]:
player = 'Daniel Jones'
g = df_player_box_scores.groupby(by="Player")
df = g.get_group(player)


In [ ]:
df

In [ ]:


def scatterplot_acquisitions(df_stints, select_acq_method=None, select_positions=None):
    
    if select_acq_method is None:
        select_acq_method = [True]

    if select_positions is None:
        select_positions=['RB','WR','TE']

    g = df_stints.groupby(by=["Drafted", "Position"])

    df = pd.concat([g.get_group((acq_by_draft,position)) 
           for acq_by_draft in select_acq_method
           for position in select_positions],axis=0)

    if select_acq_method[0]:
        status = "Draft"
    else:
        status = "Waiver"

    positions = ', '.join(select_positions)

    plot_title = f"Position: {positions} , Acquired by: {status}"
    selection = alt.selection_multi(fields=["Team"], bind="legend")

    color = alt.condition(
        selection,
        alt.Color(
            "Team:N",
            scale=alt.Scale(scheme="tableau20"),
        ),
        alt.value("lightgray"),
    )

    chart = (
        alt.Chart(df)
        .mark_circle(size=40)
        .encode(
            alt.X("Bid Amount ($)", axis=alt.Axis(grid=False)),
            alt.Y("Total points per stint", axis=alt.Axis(grid=False)),
            color=color,
            opacity=alt.condition(selection, alt.value(1), alt.value(0.1)),
            tooltip=["Player", "Team", "Bid Amount ($)", "Total points per stint"],
        )
        .add_selection(selection)
        .properties(width=450, height=450, title=plot_title)
        .configure_axis(labelFontSize=18, titleFontSize=18)
        .configure_title(fontSize=20)
        .configure_legend(labelFontSize=14, titleFontSize=14)
    )
    return plot_title, chart


## Draft Scatterplot

In [ ]:
plot_title, chart = scatterplot_acquisitions(df_stints, select_acq_method=[True], select_positions=['RB','WR','TE'])
chart.display()

## Waiver Scatterplot

- In waiver spending, Flex Player All Stars really spent a fortune and didn't get great return on Khalil Herbert. 
- Jamaal Williams, Jerick McKinnon, Curtis Samuel and D'Onta Foreman, were big value adds

In [ ]:
plot_title, chart = scatterplot_acquisitions(df_stints, select_acq_method=[False], select_positions=['RB','WR','TE'])
chart.display()

Curtis Samuel was the most valuable waiver wire receiver.

In [ ]:
for acq_by_draft in [True,False]:
    for position in df_stints['Position'].unique():
        plot_title,chart = scatterplot_acquisitions(df_stints, select_acq_method=[acq_by_draft], select_positions=[position])
        chart.display()


In [ ]:
df_player_ffteam['Position'].unique()

In [ ]:
df_player_ffteam[(df_player_ffteam['Num weeks']>7) & (df_player_ffteam['Position'].isin(['QB','RB','WR','TE'])) & (~df_player_ffteam['Drafted'])].sort_values(by='quantile',ascending=False)

In [ ]:
df_player_ffteam[(df_player_ffteam['Num weeks']>7) & (df_player_ffteam['Position'].isin(['QB','RB','WR','TE'])) &(df_player_ffteam['quantile']>=90) ].sort_values(by='quantile',ascending=False)